In [1]:
import sys
import os
from pathlib import Path

project_root = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
sys.path.insert(0, str(project_root))

from core.backtesting_opt import _Data, Strategy, Backtest
from utils.functions import BlackScholes as bs
import pandas as pd
import numpy as np
import math
from scipy.optimize import root_scalar
from datetime import datetime, date as DateObject

/opt/homebrew/Caskroom/miniconda/base/envs/backtester/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 Successfully imported local quantstats from: /Users/aniruddhapanda/Desktop/2cents/FnO-Synapse/quantstats


In [2]:
class IV_Slope(Strategy):
    upper_threshold = 0.19
    lower_threshold = 0.15
    position_id= 0
    
    def init(self):
        self.legs = {
            'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None},
            'leg2': {'type': 'PE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None},
            'leg3': {'type': 'CE', 'expiry_type': 'monthly', 'expiry_range': [26, 34], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None},
            'leg4': {'type': 'PE', 'expiry_type': 'monthly', 'expiry_range': [26, 34], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None}
        }
        # Flag to track if we've already executed today
        self.executed_today = False
        self.last_execution_date = None
        self.previous_signal = 0
    
    def execution_window(self, start_time_str="14:30:00", end_time_str="15:00:00"):
        current_time = pd.Timestamp(self.time).time()
        current_date = pd.Timestamp(self.time).date()
        
        # Reset execution flag for new trading day
        if self.last_execution_date != current_date:
            self.executed_today = False
            self.last_execution_date = current_date
        
        # Check if current time is within execution window
        start_time = pd.Timestamp(start_time_str).time()
        end_time = pd.Timestamp(end_time_str).time()
        
        if not (start_time <= current_time <= end_time):
            return False
            
        # Execute only once per day during the time window
        if self.executed_today:
            return False
            
        # Mark as executed for today
        self.executed_today = True
        return True
    
    def next(self):
        if self.spot is None or pd.isna(self.spot):
            return
            
        # Check if we should execute today
        if not self.execution_window():
            return
        
        atm = round(self.spot / 50) * 50
        
        for leg in self.legs.values():
            available_ttes = list(self.tte_to_expiry.keys()) if self.tte_to_expiry else []
            lower, upper = leg["expiry_range"]
            matching_ttes = [tte for tte in available_ttes if lower <= tte <= upper]
            if not matching_ttes:
                print(f"No matching TTEs found for leg {leg['type']} with range {leg['expiry_range']}. Available TTEs: {sorted(available_ttes) if available_ttes else 'None'}")
                return 
            
            valid_tte = min(matching_ttes)
            leg["expiry"] = self.tte_to_expiry[valid_tte]
            
            leg["strike"] = float(atm)
            contract = f"NIFTY{pd.Timestamp(leg['expiry']).strftime('%d%b%y').upper()}{int(leg['strike'])}{leg['type']}"
            leg["contract"] = contract
            leg["data"] = self._data.get_ticker_data(contract)
            
        missing_legs = [leg["contract"] for leg in self.legs.values() if leg["data"] is None]
        if missing_legs:
            print(f"IV not found for {self.time}. Spot: {self.spot} Missing legs: {missing_legs}")
            return
        
        iv_slope = math.log((self.legs["leg1"]["data"]["iv"] + self.legs["leg2"]["data"]["iv"]) / (self.legs["leg3"]["data"]["iv"] + self.legs["leg4"]["data"]["iv"]), 10)
        
        active_trades = self.active_trades
        new_signal = 0
        if iv_slope > self.upper_threshold:
            new_signal = -1
        elif iv_slope < self.lower_threshold:
            new_signal = 1
        
        if new_signal == -1:
            self.entry_type_dict = {'weekly': 'SELL', 'monthly': 'BUY'}
        elif new_signal == 1:
            self.entry_type_dict = {'weekly': 'BUY', 'monthly': 'SELL'}
        elif new_signal == 0:
            return
            
        
        if not active_trades:
            
            placed_any_leg = False
            for leg_id, leg in self.legs.items():
                entry_type = self.entry_type_dict.get(leg["expiry_type"])
                order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
                if order_fn is None:
                    continue
                order_fn(
                    strategy_id='strat1',
                    position_id=self.position_id,
                    leg_id=leg_id,
                    ticker=leg["contract"],
                    quantity=1,
                    stop_loss=None,
                    take_profit=None,
                    tag=f'{new_signal} signal entry'
                )
                placed_any_leg = True
            if placed_any_leg:
                self.position_id += 1
                self.previous_signal = new_signal
        else:
            near_expiry = None
            for trade in active_trades:
                expiry = datetime.strptime(trade.ticker[-14:-7], "%d%b%y").date()
                near_expiry = expiry if near_expiry is None else min(near_expiry, expiry)
            
            exit_reason = (
                "Near Expiry reached" if (pd.Timestamp(self.time).date() == near_expiry) else
                "Signal changed" if (self.previous_signal != new_signal) else
                None
            )
            
            if exit_reason:
                for trade in active_trades:
                    print("Closing position")
                    trade.close(trade.size, tag=exit_reason)
                self.previous_signal = 0
                
                placed_any_leg = False
                for leg_id, leg in self.legs.items():
                    entry_type = self.entry_type_dict.get(leg["expiry_type"])
                    order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
                    if order_fn is None:
                        continue
                    order_fn(
                        strategy_id='strat1',
                        position_id=self.position_id,
                        leg_id=leg_id,
                        ticker=leg["contract"],
                        quantity=1,
                        stop_loss=None,
                        take_profit=None,
                        tag=f'{new_signal} signal entry'
                    )
                    placed_any_leg = True
                if placed_any_leg:
                    self.position_id += 1
                    self.previous_signal = new_signal
                
                
            else:
                if self.previous_signal == new_signal:
                    leg_strike = self.legs["leg2"]["strike"]
                    if (0.99*leg_strike) <= self.spot <= (1.01*leg_strike):
                        pass
                    else:
                        # Close only monthly contracts (leg3 and leg4)
                        monthly_trades_closed = False
                        for trade in active_trades:
                            if trade.leg_id in ['leg3', 'leg4']:
                                print(f"Closing monthly contract: {trade.leg_id}")
                                trade.close(trade.size, tag="Monthly adjustment - lower strike")
                                monthly_trades_closed = True
                        
                        # If monthly contracts were closed, place new orders for long contracts only
                        if monthly_trades_closed:
                            print(f"Placing new long contracts at current ATM: {atm}")
                            
                            # Place orders according to entry type
                            placed_any_leg = False
                            for leg_id, leg in self.legs.items():
                                if leg_id in ['leg1', 'leg2']:
                                    continue
                                if new_signal == -1:
                                    entry_type = 'BUY'
                                elif new_signal == 1:
                                    entry_type = 'SELL'
                                else:
                                    continue
                                order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
                                if order_fn is None:
                                    continue
                                print(f"Placing {entry_type} order for {leg_id}: {leg['contract']}")
                                order_fn(
                                    strategy_id='strat1',
                                    position_id=self.position_id,
                                    leg_id=leg_id,
                                    ticker=leg["contract"],
                                    quantity=1,
                                    stop_loss=None,
                                    take_profit=None,
                                    tag=f'Adjustment {entry_type} at ATM {atm}'
                                )
                                placed_any_leg = True
                            
                            if placed_any_leg:
                                self.position_id += 1

In [ ]:
db_path = "../core/nifty_1min_desiquant.duckdb"

bt = Backtest(db_path=db_path, strategy=IV_Slope, cash=10000000, commission_per_contract=0.65, option_multiplier=75)
stats = bt.run(start_date="2022-01-01", end_date="2023-01-01")
print(stats)

In [3]:
from core.wfo import WalkForwardOptimizer

# constraint = lambda x: x['upper_threshold'] > x['lower_threshold']


wfo = WalkForwardOptimizer(
    strategy=IV_Slope,
    optimization_params={
        'upper_threshold': [-0.15 ,-0.1, 0.1, 0.15, 0.2, 0.25, 0.3],
        'lower_threshold': [-0.15 ,-0.1 ,0, 0.05, 0.1, 0.15, 0.2]
    },
    # constraint=constraint,
    maximize='Sharpe Ratio',
)

In [4]:
# def optimize_stock(self, stock, timeframe, db_path, exchange = None, training_candles = 2000, testing_candles = 200):
    
wfo.optimize_stock(
    stock='NIFTY',
    timeframe='1min',
    db_path='../core/nifty_1min_desiquant.duckdb',
    exchange='NSE',
    training_candles=6000,
    testing_candles=3000,
    start_date='2022-01-01',
    end_date='2022-12-31',
    # method='sambo',
    # max_tries=10
)

Using date range: 2022-01-01 to 2022-12-31


Walk-Forward Optimizing NIFTY:   0%|          | 0/43 [00:00<?, ?it/s]

Identifying tables in date range...
Found 248 tables out of 815 total tables with data in range
Custom Backtest.optimize called with start_date=2022-01-03 09:15:00, end_date=2022-01-24 15:30:00
Optimization parameters: {'maximize': 'Sharpe Ratio', 'constraint': <function <lambda> at 0x114b7fce0>, 'method': 'grid', 'max_tries': None, 'upper_threshold': [0.1, 0.15, 0.2, 0.25, 0.3], 'lower_threshold': [0, 0.05, 0.1, 0.15, 0.2]}
Identifying tables in date range...
Found 16 tables out of 815 total tables with data in range


IV not found for 2022-01-03 14:30:00. Spot: 17618.0 Missing legs: ['NIFTY03FEB2217600PE']


IV not found for 2022-01-04 14:30:00. Spot: 17750.0 Missing legs: ['NIFTY03FEB2217750PE']
exec_price: 224.9499969482422, order: NIFTY20JAN2217950CE, size: 1
exec_price: 218.0, order: NIFTY20JAN2217950PE, size: 1
exec_price: 452.04998779296875, order: NIFTY03FEB2217950CE, size: -1
exec_price: 877.7999877929688, order: NIFTY03FEB2217950PE, size: -1


...

exec_price: 221.75, order: NIFTY29DEC2218650CE, size: 1
exec_price: 125.75, order: NIFTY29DEC2218650PE, size: 1
exec_price: 452.8999938964844, order: NIFTY12JAN2318650CE, size: -1
exec_price: 445.8999938964844, order: NIFTY12JAN2318650PE, size: -1


Walk-Forward Optimizing NIFTY:  70%|██████▉   | 30/43 [1:26:28<39:42, 183.30s/it]

Finalizing trade for NIFTY29DEC2218650CE at price 2.0
Finalizing trade for NIFTY29DEC2218650PE at price 630.9000244140625
Finalizing trade for NIFTY12JAN2318650CE at price 32.75
Finalizing trade for NIFTY12JAN2318650PE at price 614.75


Walk-Forward Optimizing NIFTY:  70%|██████▉   | 30/43 [1:26:29<37:28, 172.97s/it]


Created default benchmark with 247 data points
No backtest object provided. Skipping equity curve generation.
No equity_df provided
HTML report saved to: /Users/aniruddhapanda/Desktop/2cents/FnO-Synapse/demos/wfo_IV_Slope_NIFTY_1min_20250627_193954/NIFTY_1min.html


In [ ]:
bt.tear_sheet()